
DOWNLOADING The library
```
# This is formatted as code
```



In [ ]:
!mkdir /content
%cd /content

In [1]:
!git clone https://github.com/PrecisionAgricultureCPP/darknet.git
%cd darknet
!git checkout PrecAgYOLO

Cloning into 'darknet'...
remote: Enumerating objects: 5905, done.
remote: Total 5905 (delta 0), reused 0 (delta 0), pack-reused 5905
Receiving objects: 100% (5905/5905), 6.16 MiB | 36.26 MiB/s, done.
Resolving deltas: 100% (3927/3927), done.
/content/darknet
Branch 'PrecAgYOLO' set up to track remote branch 'PrecAgYOLO' from 'origin'.
Switched to a new branch 'PrecAgYOLO'


Change Makefile so that Darknet compiles with CUDA

In [2]:
 !sed -i 's/GPU=0/GPU=1/' Makefile
 !cat Makefile
 

GPU=1
CUDNN=0
OPENCV=0
OPENMP=0
DEBUG=0

ARCH= -gencode arch=compute_30,code=sm_30 \
      -gencode arch=compute_35,code=sm_35 \
      -gencode arch=compute_50,code=[sm_50,compute_50] \
      -gencode arch=compute_52,code=[sm_52,compute_52]
#      -gencode arch=compute_20,code=[sm_20,sm_21] \ This one is deprecated?

# This is what I use, uncomment if you know your arch and want to specify
# ARCH= -gencode arch=compute_52,code=compute_52

VPATH=./src/:./examples
SLIB=libdarknet.so
ALIB=libdarknet.a
EXEC=darknet
OBJDIR=./obj/

CC=gcc
CPP=g++
NVCC=nvcc 
AR=ar
ARFLAGS=rcs
OPTS=-Ofast
LDFLAGS= -lm -pthread 
COMMON= -Iinclude/ -Isrc/
CFLAGS=-Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC

ifeq ($(OPENMP), 1) 
CFLAGS+= -fopenmp
endif

ifeq ($(DEBUG), 1) 
OPTS=-O0 -g
endif

CFLAGS+=$(OPTS)

ifeq ($(OPENCV), 1) 
COMMON+= -DOPENCV
CFLAGS+= -DOPENCV
LDFLAGS+= `pkg-config --libs opencv` -lstdc++
COMMON+= `pkg-config --cflags opencv` 
endif

ifeq ($(GPU), 1) 
COMMON+= -DGPU -I/u

Compiling the library


In [3]:
!make

mkdir -p obj
mkdir -p backup
mkdir -p results
gcc -Iinclude/ -Isrc/ -DGPU -I/usr/local/cuda/include/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DGPU -c ./src/gemm.c -o obj/gemm.o
./src/gemm.c: In function ‘time_gpu’:
./src/gemm.c:232:9: warning: ‘cudaThreadSynchronize’ is deprecated [-Wdeprecated-declarations]
         cudaThreadSynchronize();
         ^~~~~~~~~~~~~~~~~~~~~
In file included from /usr/local/cuda/include/cuda_runtime.h:96:0,
                 from include/darknet.h:11,
                 from ./src/utils.h:5,
                 from ./src/gemm.c:2:
/usr/local/cuda/include/cuda_runtime_api.h:947:57: note: declared here
 extern __CUDA_DEPRECATED __host__ cudaError_t CUDARTAPI cudaThreadSynchronize(void);
                                                         ^~~~~~~~~~~~~~~~~~~~~
gcc -Iinclude/ -Isrc/ -DGPU -I/usr/local/cuda/include/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DGPU -c ./src/utils.c -o obj/utils.o


downloading the tiny weights


In [4]:
!wget  https://pjreddie.com/media/files/yolov3-tiny.weights

--2019-07-31 23:57:46--  https://pjreddie.com/media/files/yolov3-tiny.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35434956 (34M) [application/octet-stream]
Saving to: ‘yolov3-tiny.weights’

yolov3-tiny.weights 100%[===================>]  33.79M  91.6MB/s    in 0.4s    

2019-07-31 23:57:46 (91.6 MB/s) - ‘yolov3-tiny.weights’ saved [35434956/35434956]



getting the pretrain weights for tiny yolo

In [5]:
!./darknet partial cfg/yolov3-tiny.cfg yolov3-tiny.weights yolov3-tiny.conv.15 15

layer     filters    size              input                output
    0 conv     16  3 x 3 / 1   416 x 416 x   3   ->   416 x 416 x  16  0.150 BFLOPs
    1 max          2 x 2 / 2   416 x 416 x  16   ->   208 x 208 x  16
    2 conv     32  3 x 3 / 1   208 x 208 x  16   ->   208 x 208 x  32  0.399 BFLOPs
    3 max          2 x 2 / 2   208 x 208 x  32   ->   104 x 104 x  32
    4 conv     64  3 x 3 / 1   104 x 104 x  32   ->   104 x 104 x  64  0.399 BFLOPs
    5 max          2 x 2 / 2   104 x 104 x  64   ->    52 x  52 x  64
    6 conv    128  3 x 3 / 1    52 x  52 x  64   ->    52 x  52 x 128  0.399 BFLOPs
    7 max          2 x 2 / 2    52 x  52 x 128   ->    26 x  26 x 128
    8 conv    256  3 x 3 / 1    26 x  26 x 128   ->    26 x  26 x 256  0.399 BFLOPs
    9 max          2 x 2 / 2    26 x  26 x 256   ->    13 x  13 x 256
   10 conv    512  3 x 3 / 1    13 x  13 x 256   ->    13 x  13 x 512  0.399 BFLOPs
   11 max          2 x 2 / 1    13 x  13 x 512   ->    13 x  13 x 512
   12 con

Clone data repo

In [6]:
!git clone https://github.com/PrecisionAgricultureCPP/Yolo_data.git

Cloning into 'Yolo_data'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 1048 (delta 12), reused 9 (delta 2), pack-reused 1020
Receiving objects: 100% (1048/1048), 1.46 GiB | 57.60 MiB/s, done.
Resolving deltas: 100% (13/13), done.
Checking out files: 100% (1017/1017), done.


Copy files into darknet directories, then remove Yolo_data directory

In [0]:
!cp -r Yolo_data/data Weeds
!cp -r Yolo_data/data Weeds_Test
!cp -r Yolo_data/'Training Weights' Weeds_Weights
!cp Yolo_data/Config/* /content/darknet/cfg
!rm -r Yolo_data

In [12]:
%cd /content/darknet
!./darknet detector train cfg/weeds.data cfg/weeds.cfg Weeds_Weights/weeds_1000.weights
#!./darknet detector test cfg/weeds.data cfg/weeds.cfg Weeds_Weights/weeds_1000.weights Yolo_data/data/MVI_7672.MOV-frame34.jpg

/content/darknet
weeds
layer     filters    size              input                output
    0 conv     16  3 x 3 / 1   416 x 416 x   3   ->   416 x 416 x  16  0.150 BFLOPs
    1 max          2 x 2 / 2   416 x 416 x  16   ->   208 x 208 x  16
    2 conv     32  3 x 3 / 1   208 x 208 x  16   ->   208 x 208 x  32  0.399 BFLOPs
    3 max          2 x 2 / 2   208 x 208 x  32   ->   104 x 104 x  32
    4 conv     64  3 x 3 / 1   104 x 104 x  32   ->   104 x 104 x  64  0.399 BFLOPs
    5 max          2 x 2 / 2   104 x 104 x  64   ->    52 x  52 x  64
    6 conv    128  3 x 3 / 1    52 x  52 x  64   ->    52 x  52 x 128  0.399 BFLOPs
    7 max          2 x 2 / 2    52 x  52 x 128   ->    26 x  26 x 128
    8 conv    256  3 x 3 / 1    26 x  26 x 128   ->    26 x  26 x 256  0.399 BFLOPs
    9 max          2 x 2 / 2    26 x  26 x 256   ->    13 x  13 x 256
   10 conv    512  3 x 3 / 1    13 x  13 x 256   ->    13 x  13 x 512  0.399 BFLOPs
   11 max          2 x 2 / 1    13 x  13 x 512   ->    1

In [0]:
#!mv Weeds/'MVI_7277.MOV-frame87 12.00.47 AM.jpg' Weeds/'MVI_7277.MOV-frame87.jpg'
#!ls Weeds | grep '.*7277.*12\.'
#!git clone https://github.com/PrecisionAgricultureCPP/Yolo_data.git
#!cp -r Yolo_data/'Training Weights' Weeds_Weights
#!echo 'Weed' > cfg/weeds.names
#!sed -i 's/names = weeds\.names/names = \/content\/darknet\/cfg\/weed\.names/' cfg/weeds.data
#!cat cfg/weeds_test.txt
#!sed -i 's/^/\/content\/darknet\//' cfg/weeds_test.txt
#!cat cfg/weeds_test.txt